In [ ]:
import os
import time
from pathlib import Path
import calendar
import earthaccess

# =====================
# CONFIG
# =====================
OUTROOT = Path("/share/home/dq076/bedrock/data/ET/MOD16A2GF_v6.1/rawdata").resolve()  # 改成你的真实路径
OUTROOT.mkdir(parents=True, exist_ok=True)

SHORT_NAME = "MOD16A2GF"
VERSION = "061"

START_YM = (2003, 1)
END_YM   = (2020, 12)

# =====================
# UTILS
# =====================
def dir_size_bytes(d: Path) -> int:
    total = 0
    for p in d.rglob("*"):
        if p.is_file():
            try:
                total += p.stat().st_size
            except FileNotFoundError:
                pass
    return total

def month_range():
    y, m = START_YM
    end_y, end_m = END_YM
    while (y < end_y) or (y == end_y and m <= end_m):
        yield y, m
        m += 1
        if m == 13:
            y += 1
            m = 1

def ensure_netrc_permissions():
    netrc = Path.home() / ".netrc"
    if not netrc.exists():
        raise FileNotFoundError(f"未找到 {netrc}，请先创建并写入 urs.earthdata.nasa.gov 的登录信息。")
    # 建议权限 600
    try:
        os.chmod(netrc, 0o600)
    except PermissionError:
        pass

def login_with_netrc():
    """
    优先使用 ~/.netrc 登录。不同版本 earthaccess 对 login 参数支持不完全一致：
    - 若支持 strategy="netrc"：用它
    - 否则：直接 earthaccess.login()（通常也会尝试 netrc）
    """
    ensure_netrc_permissions()
    try:
        # 某些版本支持
        earthaccess.login(strategy="netrc")
    except TypeError:
        # 兼容旧版本
        earthaccess.login()

def retry(func, retries=10, sleep=60):
    for i in range(1, retries + 1):
        try:
            return func()
        except Exception as e:
            print(f"[WARN] attempt {i}/{retries} failed: {e}")
            if i == retries:
                raise
            time.sleep(sleep * i)  # 递增等待（指数/线性退避）

# =====================
# DOWNLOAD
# =====================
def download_one_month(year: int, month: int):
    # 当月起止日期
    last_day = calendar.monthrange(year, month)[1]
    t_start = f"{year:04d}-{month:02d}-01"
    t_end   = f"{year:04d}-{month:02d}-{last_day:02d}"

    outdir = OUTROOT
    outdir.mkdir(parents=True, exist_ok=True)

    size0 = dir_size_bytes(outdir)
    t0 = time.time()

    granules = earthaccess.search_data(
        short_name=SHORT_NAME,
        version=VERSION,
        temporal=(t_start, t_end),
    )

    print(f"[{year:04d}-{month:02d}] temporal=({t_start},{t_end}) granules={len(granules)}")

    # 下载
    earthaccess.download(granules, local_path=str(outdir))

    t1 = time.time()
    size1 = dir_size_bytes(outdir)
    dt = max(t1 - t0, 1e-6)
    speed = (size1 - size0) / dt / (1024**2)

    print(f"[{year:04d}-{month:02d}] +{(size1-size0)/1024**3:.2f} GB in {dt/60:.1f} min, avg {speed:.2f} MB/s")


# =====================
# RUN
# =====================
login_with_netrc()

for y, m in month_range():
    retry(lambda y=y, m=m: download_one_month(y, m), retries=10, sleep=60)


/share/home/dq076/software/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2017-03] temporal=(2017-03-01,2017-03-31) granules=1435


QUEUEING TASKS | : 100%|██████████| 1435/1435 [00:00<00:00, 23584.83it/s]
PROCESSING TASKS | : 100%|██████████| 1435/1435 [00:00<00:00, 4684.64it/s]
COLLECTING RESULTS | : 100%|██████████| 1435/1435 [00:00<00:00, 247525.34it/s]


[2017-03] +0.00 GB in 0.1 min, avg 0.00 MB/s
[2017-04] temporal=(2017-04-01,2017-04-30) granules=1148


QUEUEING TASKS | : 100%|██████████| 1148/1148 [00:00<00:00, 38518.02it/s]
PROCESSING TASKS | : 100%|██████████| 1148/1148 [08:20<00:00,  2.30it/s] 
COLLECTING RESULTS | : 100%|██████████| 1148/1148 [00:00<00:00, 254832.55it/s]


...

[2020-10] +9.69 GB in 11.5 min, avg 14.32 MB/s
[2020-11] temporal=(2020-11-01,2020-11-30) granules=1132


QUEUEING TASKS | : 100%|██████████| 1132/1132 [00:00<00:00, 28785.41it/s]
PROCESSING TASKS | : 100%|██████████| 1132/1132 [08:35<00:00,  2.19it/s]
COLLECTING RESULTS | : 100%|██████████| 1132/1132 [00:00<00:00, 158381.22it/s]


[2020-11] +6.94 GB in 8.7 min, avg 13.65 MB/s
[2020-12] temporal=(2020-12-01,2020-12-31) granules=1415


QUEUEING TASKS | : 100%|██████████| 1415/1415 [00:00<00:00, 28223.85it/s]
PROCESSING TASKS | : 100%|██████████| 1415/1415 [00:23<00:00, 60.73it/s] 
COLLECTING RESULTS | : 100%|██████████| 1415/1415 [00:00<00:00, 340599.15it/s]


[2020-12] +0.32 GB in 0.5 min, avg 11.67 MB/s
